In [ ]:
import os
os.mkdir('data')

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python

In [ ]:
# !pip install llama-cpp-python
!pip install llama-index
!pip install llama-index-llms-llama-cpp
!pip install llama-index-embeddings-huggingface
!pip install -U gradio

In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (messages_to_prompt, completion_to_prompt)

llm = LlamaCPP(
    model_url='https://huggingface.co/ibanerjee/llama2-chat-quant-llm/resolve/main/Q4_K_M.gguf',
    # model_url='https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q2_K.gguf',
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
)

total size (MB): 4081.01


3892it [02:39, 24.45it/s]                          


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document, GPTVectorStoreIndex

documents = SimpleDirectoryReader(input_dir = "/content/data/").load_data()

documents = Document(text = "\n\n".join([doc.text for doc in documents]))

Failed to load file /content/data/AlanWatts-The Way of Zen.pdf with error: RetryError[<Future at 0x78f7115d4d00 state=finished raised OSError>]. Skipping...


Failed to load file /content/data/wisdom-of-insecurity.pdf with error: RetryError[<Future at 0x78f7115d57b0 state=finished raised PdfStreamError>]. Skipping...


In [ ]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import VectorStoreIndex, ServiceContext, load_index_from_storage, StorageContext, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


embedding_model = HuggingFaceEmbedding(model_name="WhereIsAI/UAE-Large-V1")


def build_index(documents, llm, embed_model=embedding_model, sentence_window_size=3, save_dir="./vector_store/index"):

    node_parser = SentenceWindowNodeParser(
        window_size = sentence_window_size,
        window_metadata_key = "window",
        original_text_metadata_key = "original_text",
        chunk_size = 256
    )

    # Deprecated
    # sentence_context = ServiceContext.from_defaults(
    #     llm=llm,
    #     embed_model= embed_model,
    #     node_parser = node_parser,
    #     chunk_size=256,
    #     context_window=3900
    # )

    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.node_parser = node_parser
    Settings.context_window = 3900


    if not os.path.exists(save_dir):
        index = VectorStoreIndex.from_documents(
            [documents], service_context=Settings
        )
        index.storage_context.persist(persist_dir=save_dir)
    else:
        index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=Settings,
        )

    return index

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/65.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
index = build_index(documents=documents, llm=llm, embed_model=embedding_model, sentence_window_size=3, save_dir="./vector_store/index")

In [ ]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

def get_engine(llm, memory, sentence_index, similarity_top_k=3, rerank_top_n=2):
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    c_engine = sentence_index.as_chat_engine(
        chat_mode = 'context',
        llm = llm,
        memory = memory,
        system_prompt=(
            "You are a chatbot, capable of engaging in normal interactions and philosophical discussions."
            "For greetings like 'Hi', 'Hello', and 'How are you', respond with greetings without searching."
            "Provide philosophical replies only when explicitly asked."
            "Offer information about the philosophies of Alan Watts only when queried."
            "Maintain a professional and wise tone, embodying the role of a mentor."
            "Your goal is to provide insightful and meaningful responses tailored to the user's queries."
            "Keep your responses brief and to the point."
        ),
        similarity_top_k = similarity_top_k,
        node_postprocessors = [postproc, rerank],
        streaming=True
    )


    return c_engine

In [ ]:
chat_engine = get_engine(llm, memory, sentence_index=index, similarity_top_k=3, rerank_top_n=2)

In [ ]:
# from IPython.display import HTML, display

# def set_css():
#   display(HTML('''
#   <style>
#     pre {
#         white-space: pre-wrap;
#     }
#   </style>
#   '''))
# get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# first_click = True

# while True:
#     # query = input("User: ")
#     if first_click:
#         response = chat_engine.chat("Hi")
#         first_click = False
#         continue

#     query = input("User: ")
#     response = chat_engine.stream_chat(query)

#     print("Response:", end="")
#     for token in response.response_gen:
#         print(token, end="")

#     print("\n\n\n\n\n")

In [ ]:
import time
import gradio as gr

seed_response = chat_engine.chat("Hi")

examples = ["What does it mean to be human?",
            "How can we have peace?",
            "Tell me 3 steps for self improvement",
            "Tell me about basic human behavior"]

def chatbot(query, history):
    response = chat_engine.stream_chat(query)
    resp = ""

    for token in response.response_gen:
        resp+=token
        time.sleep(0.1)
        yield resp


def reset_memory():
    chat_engine.reset()
    seed_response = chat_engine.chat("Hi")


In [ ]:
gr.ChatInterface(
    fn = chatbot,
    title = "Chatbot using Llama index and Llama2 GGUF",
    examples = examples
).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3a17cc2b1df65d2f7a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# def gradio_chat():

#     with gr.Blocks() as demo:
#         gr.Markdown("# Chatbot using Llama index and Llama2 GGUF")
#         with gr.Row():
#             with gr.Column():
#                 user_input = gr.Textbox(label="Query")
#                 submit_button = gr.Button("Submit")
#                 reset_button = gr.Button("Clear chat memory")
#             with gr.Column():
#                 output = gr.Textbox(label="Response")

#         submit_button.click(chatbot, [user_input], output)
#         reset_button.click(reset_memory)

#     demo.launch()

# if __name__ == "__main__":
#     gradio_chat()

In [ ]:
# chat_engine.reset()